## 模型训练

- 数据集切分按照疾病、种族进行

- 目前只针对蛋白组

- 分种族训练



### 预处理

In [1]:
from config import *
import pandas as pd
import json
from ppp_prediction.utils import load_data
from ppp_prediction.model import fit_best_model
import pickle
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
from ppp_prediction.plot.utils import save_fig

/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2025-02-19 11:44:56,426	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-19 11:44:56,552	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-19 11:44:56,626	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# Define Basic Variables

groupByVar = "Ethnic"  # Ethnic
omicsName = "Prot_meanImpute"  # used omics Name
phenoDefineVersion = "Lancet_Digital_Health_2019"  # used pheno version

## cutoff
Case_cutoff = 50  # only over this number of cases will be used as a phenotype

In [3]:
# Define the dirs
covariates_dir = dataDir / "covariates.feather"
omicsDataDir = dataDir / f"Prot/{omicsName}.feather"

phenoDefineDir = dataDir / f"{phenoDefineVersion}"

In [4]:
# load data
covariates_df = pd.read_feather(covariates_dir)
omicsData = pd.read_feather(omicsDataDir)
print(
    f"Total {omicsData.shape[0]} samples and {omicsData.shape[1]} features with {omicsName}"
)
diseaseList = list(phenoDefineDir.glob("*.feather"))
foundedPhenoFile = len(list(phenoDefineDir.glob("*.feather")))
print(f"Founded Pheno Files: {foundedPhenoFile}")
covariates_df

Total 53021 samples and 2912 features with Prot_meanImpute
Founded Pheno Files: 169


,eid,sex,height,weight,BSA,age,PC1,PC2,PC3,PC4,...,drug_ldl,drug_hdl,drug_tc,drug_sbp,drug_dbp,genotype_array,assessment_center,ancestry,ancestry_high_confi,Ethnic
0,1000017,1.0,172.0,89.5,2.067876,56.0,-11.3690,3.56718,-1.975530,0.213937,...,0.0,0.0,0.0,1.0,1.0,1,1,EUR,EUR,White
1,1000025,1.0,176.0,113.9,2.359755,62.0,-12.1620,2.77470,0.175048,2.554930,...,1.0,0.0,1.0,1.0,1.0,2,2,EUR,EUR,White
2,1000038,1.0,179.5,112.2,2.365252,60.0,-12.8698,6.41566,-5.106100,-1.296310,...,0.0,0.0,0.0,1.0,1.0,1,3,EUR,EUR,White
3,1000042,1.0,169.0,77.1,1.902476,60.0,72.9437,-109.21600,74.692200,17.863400,...,1.0,0.0,1.0,1.0,1.0,2,3,SAS,SAS,Asian
4,1000056,0.0,163.0,72.0,1.805547,65.0,-10.7174,5.77507,0.620341,0.505251,...,0.0,0.0,0.0,0.0,0.0,2,4,EUR,EUR,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502404,6024086,0.0,154.0,88.0,1.940218,66.0,-11.1845,4.08367,-0.006942,-0.325017,...,0.0,0.0,0.0,0.0,0.0,2,8,EUR,EUR,White
502405,6024098,1.0,173.0,81.3,1.976592,68.0,-13.3426,2.56658,-0.076882,6.048100,...,0.0,1.0,0.0,1.0,1.0,2,9,EUR,EUR,White
502406,6024103,1.0,187.0,111.3,2.404458,61.0,-12.2113,4.22902,-2.629170,4.489250,...,0.0,0.0,0.0,1.0,1.0,2,20,EUR,EUR,White
502407,6024110,1.0,177.0,73.2,1.897103,66.0,-10.5527,6.84118,-2.149580,-0.825010,...,0.0,0.0,0.0,0.0,0.0,2,11,EUR,EUR,White


In [5]:
# groupByVar used
used_groupByVar = ["White", "Asian", "Black"]
covariates_df[groupByVar].value_counts()

Ethnic
White      472610
Asian        9879
Black        8058
Other        7335
Mixed        2954
Chinese      1573
Name: count, dtype: int64

In [6]:
# show all disease rate in Prot


res_dict = {
    "event": [],
    "incident": [],
    "prevalent": [],
}
for disease in tqdm(diseaseList, total=len(diseaseList), desc="Counting..."):
    df = pd.read_feather(disease).query("eid in @omicsData.eid")

    for col in ["event", "incident", "prevalent"]:
        case = int(df[col].sum())
        control = int(df.shape[0] - case)
        rate = case / df.shape[0]
        res_dict[col].append(
            pd.DataFrame(
                {
                    "Phenotype": [disease.stem],
                    "Case": [case],
                    "Control": [control],
                    "Rate": [rate],
                }
            )
        )

Counting...:  50%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                            | 85/169 [00:30<00:29,  2.83it/s]


KeyboardInterrupt: 

In [ ]:
event_df = (
    pd.concat(res_dict["event"])
    .sort_values("Rate", ascending=False)
    .reset_index(drop=True)
)
incident_df = (
    pd.concat(res_dict["incident"])
    .sort_values("Rate", ascending=False)
    .reset_index(drop=True)
)
prevalent_df = (
    pd.concat(res_dict["prevalent"])
    .sort_values("Rate", ascending=False)
    .reset_index(drop=True)
)

event_df

In [ ]:
# plot
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# histplot
sns.histplot(data=event_df, x="Rate", ax=ax1, bins=50)
sns.histplot(data=incident_df, x="Rate", ax=ax2, bins=50)
sns.histplot(data=prevalent_df, x="Rate", ax=ax3, bins=50)

ax1.set_title("Event")
ax2.set_title("Incident")
ax3.set_title("Prevalent")

ax1.set_xlim(0, 0.2)
ax2.set_xlim(0, 0.2)
ax3.set_xlim(0, 0.2)

In [9]:
# event_df =
incident_df = incident_df.query("Case > @Case_cutoff")
prevalent_df = prevalent_df.query("Case > @Case_cutoff")

print(f"Finnal Incident Phenotype: {incident_df.shape[0]}")
print(f"Finnal Prevalent Phenotype: {prevalent_df.shape[0]}")

Finnal Incident Phenotype: 13
Finnal Prevalent Phenotype: 6


### Run Assoc

1. For prevalence, by Cox 
2. For incident, by logit

### 过滤表型

保存Case 至少>50的疾病

分种族过滤！

In [11]:
from ppp_prediction.model_v2.models import (
    fit_best_model_v2,
    fit_ensemble_model_simple_v2,
    fit_lightgbm,
    fit_xgboost,
)


def get_predict_v2_from_df(
    model,
    data,
    x_var,
):
    """
    merge by idx
    """

    no_na_data = data[x_var].dropna().copy()
    if hasattr(model, "predict_proba"):
        no_na_data["pred"] = model.predict_proba(no_na_data)[:, 1]
    else:
        no_na_data["pred"] = model.predict(no_na_data)

    return (
        data[[]]
        .merge(no_na_data[["pred"]], left_index=True, right_index=True, how="left")
        .values.flatten()
    )

In [12]:
from sklearn.model_selection import train_test_split
from ppp_prediction.metrics import cal_binary_metrics


# define a function to fit the model, save the result and collect the scores
## parallel this function
def fit_model_and_save_result(
    total_df,  # first col should be eid
    label_df,  # first col should be eid
    save_dir,
    # asssoc_df=None, # used to sort and get top features to downsample, if None then will do assoc
    feature_rank_list=None,  # used to downsample the features
    min_class_number_cutoff={"train": 30, "validation": 10, "test": 10},
    train_test_split_ratio=0.7,
    seed=1234,
    device="cuda",
    topk_list =[5, 10, 20, 50, 100] 
):
    """
    1. merge the total_df and disease_df
    2. check label_df sum is over min_class_number_cutoff; if not return None, and print the error
    3. fit the models: 1) full lasso, 2) full xgboost, 3) sample lasso, 4) sample xgboost, 5) sample AutoTabPFN, 6) TabPFN
    4. save the results: 1) model, 2) scores of total_df, 3) metrics of models
    """
    # step1: merge and check

    ## check the first column
    if total_df.columns[0] != "eid":
        raise ValueError("total_df first column should be eid")
    if label_df.columns[0] != "eid":
        raise ValueError("label_df first column should be eid")

    features = total_df.columns[1:].tolist()
    label = label_df.columns[1]

    ## merge
    merged_df = pd.merge(total_df, label_df, on="eid", how="inner")
    print(
        f"Found merged samples: {merged_df.shape[0]} while, label_df: {label_df.shape[0]} and total_df: {total_df.shape[0]}"
    )

    # step2: fit the model
    ## step2.1 Train Test Split
    train_df, test_df = train_test_split(
        merged_df, test_size=1 - train_test_split_ratio, random_state=seed
    )
    train_df, val_df = train_test_split(train_df, test_size=0.3, random_state=seed)

    score_df = pd.concat(
        [
            train_df[["eid", label]].copy().assign(Type="train"),
            val_df[["eid", label]].copy().assign(Type="validation"),
            test_df[["eid", label]].copy().assign(Type="test"),
        ]
    )
    ## step2.2 check the min_class_number
    for min_class_number_check_key in ["train", "validation", "test"]:
        if min_class_number_check_key == "train":
            to_check_df = train_df
        elif min_class_number_check_key == "validation":
            to_check_df = val_df
        elif min_class_number_check_key == "test":
            to_check_df = test_df
        else:
            raise ValueError(
                "min_class_number_check_key should be in ['train', 'validation', 'test']"
            )
        min_class_number = min(to_check_df[label].value_counts())
        # the min class number and class name
        min_class_name = to_check_df[label].value_counts().idxmin()
        if min_class_number < min_class_number_cutoff[min_class_number_check_key]:
            print(
                f"Error: {min_class_number_check_key} {min_class_name} has only {min_class_number} samples, less than {min_class_number_cutoff[min_class_number_check_key]}"
            )
            return None
    print(
        f"Train data have {train_df.shape[0]} samples with {train_df[label].sum():.0f} cases"
    )
    print(
        f"Validation data have {val_df.shape[0]} samples with {val_df[label].sum():.0f} cases"
    )
    print(
        f"Test data have {test_df.shape[0]} samples with {test_df[label].sum():.0f} cases"
    )

    ## check
    train_meta_info = {}
    ## step2.4 fit the models
    modelSaveDir = save_dir / "models"
    modelSaveDir.mkdir(parents=True, exist_ok=True)
    ### 1) Lasso full
    lasso_full_savedir = modelSaveDir / "lasso_full.pkl"
    if lasso_full_savedir.exists():
        lasso_full = pickle.load(open(lasso_full_savedir, "rb"))
        # print(f"lasso_full loaded")

    else:
        lasso_engine = "cuml" if device == "cuda" else "sklearn"
        print(merged_df.shape)
        if merged_df.shape[0]  < 5000 :
            lasso_engine = "sklearn"
        print(f"lasso_full start with engine {lasso_engine}")
        (lasso_full, *_) = fit_best_model(
            train_df=train_df,
            test_df=val_df,
            X_var=features,
            y_var=label,
            method_list="Lasso",
            cv=5,
            engine=lasso_engine,
        )

        pickle.dump(lasso_full, open(lasso_full_savedir, "wb"))

    score_df["lasso_full"] = get_predict_v2_from_df(lasso_full, total_df, features)
    # return lasso_full, score_df
    train_meta_info[f"lasso_full"] = {
        "train_case": train_df[label].sum(),
        "train_control": train_df.shape[0] - train_df[label].sum(),
    }
    del lasso_full

    # xgboost full
    xgboot_full_savedir = modelSaveDir / "xgboost_full.pkl"
    if (modelSaveDir / "xgboost_full.pkl").exists():
        print(f"xgboost_full loaded")
        xgboost_full_tuned = pickle.load(open(xgboot_full_savedir, "rb"))
    else:

        xgboost_full_tuned, *_ = fit_xgboost(
            train=train_df,
            xvar=features,
            label=label,
            tuning=True,
            tune_config={"max_iter": 100},
        )
        pickle.dump(xgboost_full_tuned, open(xgboot_full_savedir, "wb"))
    score_df["xgboost_full"] = get_predict_v2_from_df(
        xgboost_full_tuned, total_df, features
    )
    train_meta_info[f"xgboost_full"] = {
        "train_case": train_df[label].sum(),
        "train_control": train_df.shape[0] - train_df[label].sum(),
    }

    del xgboost_full_tuned


    for strata in ["balance", "random"]:  # balance or random
        if strata == "balance":
            disease_train_case = train_df.query(f"{label} == 1")
            disease_train_control = train_df.query(f"{label} == 0").sample(
                n=disease_train_case.shape[0], random_state=seed
            )
            disease_train_sample = pd.concat(
                [disease_train_case, disease_train_control]
            )
        elif strata == "random":
            if train_df.shape[0] > 10000:
                disease_train_sample = train_df.sample(
                    n=10000, random_state=seed
                )
            else:
                disease_train_sample = train_df # TODO: Anno this with no sample
                
        for topk in topk_list:
            sig_features = feature_rank_list[:topk]


            suffix_name = f"{topk}_{strata}"

            print(suffix_name)

            strata_topk_save_dir = modelSaveDir / f"{topk}/{strata}"
            strata_topk_save_dir.mkdir(parents=True, exist_ok=True)

            X_train = disease_train_sample[sig_features]
            y_train = disease_train_sample[label]

            lasso_sample_topk_savedir = strata_topk_save_dir / f"lasso_sample.pkl"
            if lasso_sample_topk_savedir.exists():
                lasso_sample = pickle.load(open(lasso_sample_topk_savedir, "rb"))
                print(f"lasso_sample_{suffix_name} loaded")
            else:
                try:
                    lasso_engine = "cuml" if device == "cuda" else "sklearn"
                    if X_train.shape[0] < 5000:
                        lasso_engine = "sklearn"
                    print(f"lasso_full start with engine {lasso_engine}")

                    (lasso_sample, *_) = fit_best_model(
                        train_df=disease_train_sample,
                        test_df=val_df,
                        X_var=sig_features,
                        y_var=label,
                        method_list="Lasso",
                        cv=5,
                        engine=lasso_engine,
                    )
                    pickle.dump(
                        lasso_sample,
                        open(strata_topk_save_dir / f"lasso_sample.pkl", "wb"),
                    )
                except Exception as e:
                    print(f"lasso_sample_{topk} failed and erros: {e}")

            score_df[f"lasso_sample_{suffix_name}"] = get_predict_v2_from_df(
                lasso_sample, total_df, sig_features
            )
            train_meta_info[f"lasso_sample_{suffix_name}"] = {
                "train_case": disease_train_sample[label].sum(),
                "train_control": disease_train_sample.shape[0] - disease_train_sample[label].sum(),
            }

            del lasso_sample

            from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import (
                AutoTabPFNClassifier,
            )

            AutoTabPFN_topk_savedir = strata_topk_save_dir / f"AutoTabPFN.pkl"
            if AutoTabPFN_topk_savedir.exists():
                AutoTabPFN = pickle.load(open(AutoTabPFN_topk_savedir, "rb"))
                print(f"AutoTabPFN_{topk} loaded")
            else:

                try:
                    AutoTabPFN = AutoTabPFNClassifier(
                        max_time=120, device="cuda" if device == "cuda" else "cpu"
                    )  # 120 seconds tuning time
                    AutoTabPFN.fit(X_train, y_train)
                    pickle.dump(
                        AutoTabPFN, open(strata_topk_save_dir / f"AutoTabPFN.pkl", "wb")
                    )
                except:
                    print(f"AutoTabPFN_{topk} failed")

            # score_df["AutoTabPFN"] = AutoTabPFN.predict_proba(X_held_out_test)[:, 1]
            score_df[f"AutoTabPFN_{suffix_name}"] = AutoTabPFN.predict_proba(
                total_df[sig_features]
            )[:, 1]
            train_meta_info[f"AutoTabPFN_{suffix_name}"] = {
                "train_case": disease_train_sample[label].sum(),
                "train_control": disease_train_sample.shape[0] - disease_train_sample[label].sum(),
            }

            del AutoTabPFN
            try:
                from tabpfn import TabPFNClassifier

                TabPFN_topk_savedir = strata_topk_save_dir / f"TabPFN{topk}.pkl"
                if TabPFN_topk_savedir.exists():
                    TabPFN = pickle.load(open(TabPFN_topk_savedir, "rb"))
                    print(f"TabPFN_{topk} loaded")
                else:

                    TabPFN = TabPFNClassifier(
                        device="cuda:0" if device == "cuda" else "cpu", 
                        ignore_pretraining_limits=True
                    )
                    TabPFN.fit(X_train, y_train)
                    pickle.dump(
                        TabPFN, open(strata_topk_save_dir / f"TabPFN.pkl", "wb")
                    )
                # score_df["AutoTabPFN"] = AutoTabPFN.predict_proba(X_held_out_test)[:, 1]
                score_df[f"TabPFN_{suffix_name}"] = TabPFN.predict_proba(
                    total_df[sig_features]
                )[:, 1]
                train_meta_info[f"TabPFN_{suffix_name}"] = {
                    "train_case": disease_train_sample[label].sum(),
                    "train_control": disease_train_sample.shape[0] - disease_train_sample[label].sum(),
                }
                del TabPFN
            except:
                print(f"TabPFN {topk} failed")

            # xgboost sampled
            xgboost_sample_savedir = strata_topk_save_dir / f"xgboost_sample.pkl"
            if xgboost_sample_savedir.exists():
                xgboost_sample_tuned = pickle.load(open(xgboost_sample_savedir, "rb"))
                print(f"xgboost_sample_{topk} loaded")
            else:
                try:
                    xgboost_sample_tuned, *_ = fit_xgboost(
                        train=disease_train_sample,
                        xvar=sig_features,
                        label=label,
                        tuning=True,
                        tune_config={"max_iter": 100},
                    )
                    pickle.dump(
                        xgboost_sample_tuned,
                        open(strata_topk_save_dir / f"xgboost_sample.pkl", "wb"),
                    )
                except Exception as e:
                    print(f"xgboost_sample_{topk} failed")

            score_df[f"xgboost_sample_{suffix_name}"] = get_predict_v2_from_df(
                xgboost_sample_tuned, total_df, sig_features
            )
            train_meta_info[f"xgboost_sample_{suffix_name}"] = {
                "train_case": disease_train_sample[label].sum(),
                "train_control": disease_train_sample.shape[0] - disease_train_sample[label].sum(),
            }

            del xgboost_sample_tuned

    score_df.to_feather(save_dir / "held_out_test.feather")
    pickle.dump(train_meta_info, open(save_dir / "train_meta_info.pkl", "wb"))
    metrics_list = []
    for key in score_df.columns[3:]: # eid label Type
        to_cal_df = score_df.query("Type == 'test'")[["eid", label, key]].copy().dropna()
        res = cal_binary_metrics(
            to_cal_df[label], to_cal_df[key], n_resamples=30, ci=True
        )
        # res = run_cox(to_cal_df, var=key, E=E, T=T, ci=True, n_resamples=100)
        res["method"] = key
        res.update(train_meta_info[key])
        metrics_list.append(res)
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.to_csv(save_dir / "metrics.csv", index=False)

In [13]:
resultDir = outputDir / "test"

In [14]:
fake_rank_list = pd.read_csv(
    "../ppp/cvmdPrediction/V2/output/01-assoc/Prot/Hypertension.tsv",
    sep="\t",
)
fake_rank_list.sort_values("pvalue", inplace=True)
fake_rank_list = fake_rank_list.query("pvalue <1e-5")
fake_rank_list = fake_rank_list[["var", "coef"]]
fake_rank_list["abs_coef"] = fake_rank_list["coef"].abs()
fake_rank_list.sort_values("abs_coef", ascending=False, inplace=True)
fake_rank_list = fake_rank_list["var"].tolist()
fake_rank_list

['LEP',
 'HAVCR1',
 'PALM2',
 'IGSF9',
 'PRAP1',
 'NT5C1A',
 'HGF',
 'FURIN',
 'IL1RN',
 'FABP4',
 'CPM',
 'GDF15',
 'ADM',
 'YAP1',
 'ADAMTSL2',
 'PRSS8',
 'IFI30',
 'RBP5',
 'LGALS9',
 'CA14',
 'FABP3',
 'PON3',
 'ASGR1',
 'SERPINF1',
 'HSPB6',
 'IL6',
 'APOF',
 'INHBC',
 'FSTL3',
 'RBFOX3',
 'GPD1',
 'PCBD1',
 'MXRA8',
 'GUSB',
 'APCS',
 'CTSD',
 'GGT1',
 'FAM20A',
 'F9',
 'FGF21',
 'CDHR2',
 'FTCD',
 'ACY1',
 'PGF',
 'CFI',
 'HEG1',
 'PLAT',
 'NHLRC3',
 'CHCHD10',
 'CES1',
 'IGFBP2',
 'RARRES2',
 'HJV',
 'GLA',
 'KRT18',
 'IGFBP4',
 'TNFRSF10A',
 'PTPRB',
 'GSTA1',
 'ACE2',
 'BPIFB2',
 'SELE',
 'PRCP',
 'CDH2',
 'MSR1',
 'C3',
 'OCLN',
 'NPL',
 'REN',
 'VWA1',
 'RIDA',
 'AFM',
 'CKB',
 'SETMAR',
 'UPB1',
 'RTN4R',
 'CD80',
 'LGALS3BP',
 'ADH4',
 'LMNB2',
 'PLAUR',
 'ERBB2',
 'SEPTIN8',
 'ACTA2',
 'PLA2G15',
 'TNFRSF10B',
 'IL18R1',
 'ICAM1',
 'FABP1',
 'CCL7',
 'OXT',
 'PAMR1',
 'CRIP2',
 'SULT2A1',
 'RNASE6',
 'NPC2',
 'C1RL',
 'ADGRG1',
 'WFIKKN2',
 'SCLY',
 'RELT',
 'IGFBPL1',
 

In [15]:
# for disease in diseaseList:
for disease in [Path("V1/data/Lancet_Digital_Health_2019/hypertension.feather")]:
    diseaseName = disease.stem
    # for c_groupbyVar in used_groupByVar:
    for c_groupbyVar in ["Asian"]:
        # for label in ["prevalent", "incident"]:
        c_groupbyVar_eids = covariates_df.query(f"{groupByVar} == @c_groupbyVar").eid
        for label in ["incident", "prevalent"]:
            print(f"Currently, Running {label} {diseaseName} and {c_groupbyVar}")
            label_df = pd.read_feather(disease).query("eid in @omicsData.eid")
            fit_model_and_save_result(
                total_df=omicsData.query(f"eid in @c_groupbyVar_eids"),
                label_df=label_df[["eid", label]],
                save_dir=resultDir / f"{diseaseName}/{c_groupbyVar}/{label}",
                feature_rank_list=fake_rank_list,  # currently use the fake rank list, better use assoc on whole set or run assoc on the train set
                device="cuda",
                topk_list = [5,10,20]
            )
            break
        break
    break

Currently, Running incident hypertension and Asian
Found merged samples: 986 while, label_df: 53021 and total_df: 986
Train data have 483 samples with 136 cases
Validation data have 207 samples with 54 cases
Test data have 296 samples with 81 cases
xgboost_full loaded
5_balance
lasso_sample_5_balance loaded
AutoTabPFN_5 loaded


/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.


xgboost_sample_5 loaded
10_balance
lasso_sample_10_balance loaded
AutoTabPFN_10 loaded


/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.


xgboost_sample_10 loaded
20_balance
lasso_sample_20_balance loaded
AutoTabPFN_20 loaded


...

xgboost_sample_5 loaded
10_random
lasso_sample_10_random loaded
AutoTabPFN_10 loaded


/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.


xgboost_sample_10 loaded
20_random
lasso_sample_20_random loaded
AutoTabPFN_20 loaded


/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
/deeplearning/xutingfeng/miniforge3/envs/ml/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.


xgboost_sample_20 loaded


In [16]:
pd.read_pickle("V1/output/test/hypertension/Asian/incident/train_meta_info.pkl")

{'lasso_full': {'train_case': 136, 'train_control': 347},
 'xgboost_full': {'train_case': 136, 'train_control': 347},
 'lasso_sample_5_balance': {'train_case': 136, 'train_control': 136},
 'AutoTabPFN_5_balance': {'train_case': 136, 'train_control': 136},
 'TabPFN_5_balance': {'train_case': 136, 'train_control': 136},
 'xgboost_sample_5_balance': {'train_case': 136, 'train_control': 136},
 'lasso_sample_10_balance': {'train_case': 136, 'train_control': 136},
 'AutoTabPFN_10_balance': {'train_case': 136, 'train_control': 136},
 'TabPFN_10_balance': {'train_case': 136, 'train_control': 136},
 'xgboost_sample_10_balance': {'train_case': 136, 'train_control': 136},
 'lasso_sample_20_balance': {'train_case': 136, 'train_control': 136},
 'AutoTabPFN_20_balance': {'train_case': 136, 'train_control': 136},
 'TabPFN_20_balance': {'train_case': 136, 'train_control': 136},
 'xgboost_sample_20_balance': {'train_case': 136, 'train_control': 136},
 'lasso_sample_5_random': {'train_case': 136, 'train

In [93]:
score_df = pd.read_feather("V1/output/test/hypertension/Asian/incident/held_out_test.feather")
metrics_list = []
for key in score_df.columns[3:]:
    to_cal_df = score_df.query("Type == 'test'")[["eid", label, key]].copy().dropna()
    res = cal_binary_metrics(
        to_cal_df[label], to_cal_df[key], n_resamples=30, ci=True
    )
    # res = run_cox(to_cal_df, var=key, E=E, T=T, ci=True, n_resamples=100)
    res["method"] = key
    metrics_list.append(res)

In [94]:
score_df

,eid,incident,Type,lasso_full,xgboost_full,lasso_sample_5_balance,AutoTabPFN_5_balance,TabPFN_5_balance,xgboost_sample_5_balance,lasso_sample_10_balance,...,TabPFN_5_random,xgboost_sample_5_random,lasso_sample_10_random,AutoTabPFN_10_random,TabPFN_10_random,xgboost_sample_10_random,lasso_sample_20_random,AutoTabPFN_20_random,TabPFN_20_random,xgboost_sample_20_random
755,4965815,0,train,-0.092701,0.292974,0.489898,0.526283,0.516403,0.502275,0.414261,...,0.296645,0.293855,0.293136,0.278459,0.281512,0.301346,0.248743,0.223627,0.248202,0.281330
766,5027923,1,train,0.377075,0.292899,0.328333,0.391850,0.378329,0.501991,0.293440,...,0.150532,0.283658,0.215500,0.163865,0.162240,0.293866,0.214128,0.162959,0.167436,0.252588
101,1493006,0,train,0.227678,0.293577,0.608341,0.527133,0.568381,0.502429,0.562029,...,0.331318,0.293165,0.325195,0.320722,0.319020,0.298950,0.380376,0.363825,0.370440,0.329644
208,2069763,0,train,0.140574,0.292864,0.507239,0.508844,0.496955,0.502460,0.424892,...,0.260455,0.318641,0.277791,0.255335,0.271815,0.304927,0.245602,0.254672,0.238606,0.306260
57,1238126,0,train,0.526148,0.293153,0.437103,0.539578,0.497289,0.502277,0.433640,...,0.267647,0.279121,0.266707,0.260187,0.269442,0.300437,0.309659,0.270176,0.283464,0.269230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,4554890,1,test,0.397083,0.293353,0.590334,0.521645,0.557839,0.502628,0.552649,...,0.331240,0.281947,0.313858,0.313708,0.317855,0.311196,0.300511,0.299060,0.306942,0.318519
96,1446008,0,test,0.007065,0.292452,0.265654,0.437533,0.355252,0.501954,0.297122,...,0.137325,0.278776,0.200939,0.151250,0.167215,0.295600,0.184728,0.139354,0.149213,0.275370
927,5749777,0,test,-0.051586,0.292444,0.555780,0.525538,0.538814,0.502391,0.570118,...,0.306804,0.279121,0.298739,0.296500,0.296195,0.302735,0.310842,0.263840,0.281228,0.294891
218,2142357,0,test,0.642048,0.293347,0.453031,0.515100,0.498237,0.502462,0.423815,...,0.278600,0.294503,0.281022,0.271331,0.277141,0.300253,0.300726,0.280095,0.302931,0.289849


In [95]:
pd.DataFrame(metrics_list)

,AUC,AUC_UCI,AUC_LCI,ACC,ACC_UCI,ACC_LCI,Macro_F1,Macro_F1_UCI,Macro_F1_LCI,Sensitivity,...,Specificity,Specificity_UCI,Specificity_LCI,APR,APR_UCI,APR_LCI,N,N_case,N_control,method
0,0.502268,0.555907,0.436467,0.439189,0.469341,0.407517,0.438548,0.476080,0.373261,0.740741,...,0.325581,0.365430,0.267458,0.283973,0.324844,0.167140,296,81,215,lasso_full
1,0.507063,0.555447,0.441243,0.462838,0.500000,0.402618,0.457632,0.498329,0.391520,0.666667,...,0.386047,0.441160,0.335434,0.273266,0.345590,0.231252,296,81,215,xgboost_full
2,0.547287,0.592940,0.495733,0.408784,0.461318,0.349155,0.405793,0.461264,0.356864,0.876543,...,0.232558,0.271467,0.188014,0.290790,0.339548,0.218660,296,81,215,lasso_sample_5_balance
3,0.553431,0.626756,0.471127,0.712838,0.766301,0.675676,0.541336,0.629057,0.502667,0.185185,...,0.911628,0.948561,0.883860,0.312638,0.356771,0.248548,296,81,215,AutoTabPFN_5_balance
4,0.559575,0.635579,0.494821,0.672297,0.707010,0.612669,0.552821,0.602721,0.504395,0.283951,...,0.818605,0.876812,0.789968,0.300019,0.378804,0.239189,296,81,215,TabPFN_5_balance
5,0.503417,0.557347,0.415408,0.442568,0.485051,0.398311,0.441490,0.498245,0.374552,0.728395,...,0.334884,0.390921,0.265208,0.308282,0.395410,0.232654,296,81,215,xgboost_sample_5_balance
6,0.539994,0.592860,0.450866,0.510135,0.548226,0.465287,0.496333,0.541157,0.440416,0.629630,...,0.465116,0.540851,0.397833,0.288991,0.347849,0.251865,296,81,215,lasso_sample_10_balance
7,0.560609,0.629105,0.505568,0.385135,0.409713,0.336909,0.375904,0.424995,0.332471,0.925926,...,0.181395,0.209535,0.143181,0.297760,0.355634,0.243873,296,81,215,AutoTabPFN_10_balance
8,0.555268,0.622460,0.485252,0.587838,0.635473,0.539274,0.539388,0.597022,0.460949,0.481481,...,0.627907,0.687325,0.583244,0.290135,0.320555,0.222659,296,81,215,TabPFN_10_balance
9,0.535228,0.608521,0.481197,0.516892,0.562416,0.456926,0.504397,0.566195,0.459484,0.654321,...,0.465116,0.523781,0.426546,0.290264,0.343983,0.210833,296,81,215,xgboost_sample_10_balance


In [88]:
pd.DataFrame([model.feature_names_in_, model[-1].coef_]).T

,0,1
0,A1BG,0.002819
1,AAMDC,0.004625
2,AARSD1,0.002607
3,ABCA2,-0.000706
4,ABHD14B,0.001628
...,...,...
2906,ZNRD2,-0.004057
2907,ZNRF4,-0.003088
2908,ZP3,-0.00232
2909,ZP4,0.000005
